In [1]:
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D,\
    UpSampling2D, Lambda, Activation, Flatten, Reshape, merge
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from PIL import Image
import scipy.misc
import matplotlib.pyplot as plt
from scipy.misc import imsave
import bcolz

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test_, y_test_) = mnist.load_data()
#x_train = bcolz.open('data/x_train.bc')[:]  #FOR LSUN DATASET
x_train = (x_train.astype(np.float32)/255.0) #* 2.0 - 1.0
x_train = np.expand_dims(x_train, axis=3)

In [3]:
discriminator_model = Sequential()
discriminator_model.add(Conv2D(256, (5,5), strides=(2,2), padding='same', 
                  activation=LeakyReLU(), input_shape=(28, 28,1)))
discriminator_model.add(Conv2D(512, (5,5), strides=(2,2), padding='same', 
                  activation=LeakyReLU()))
discriminator_model.add(Flatten())
discriminator_model.add(Dense(256, activation=LeakyReLU()))
discriminator_model.add(Dense(1, activation = LeakyReLU()))
discriminator_model.compile(Adam(), loss="binary_crossentropy")

/usr/local/lib/python3.5/dist-packages/keras/activations.py:103: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  ).format(identifier=identifier.__class__.__name__))


In [4]:
generator_model = Sequential()
#generator_model.add(Dense(512*28*28, activation='relu', input_dim=100))
generator_model.add(Dense(512*7*7, activation='relu', input_dim=100))
generator_model.add(BatchNormalization())
#generator_model.add(Reshape((28, 28, 512)))
generator_model.add(Reshape((7, 7, 512)))
generator_model.add(UpSampling2D())
generator_model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
generator_model.add(BatchNormalization())
generator_model.add(UpSampling2D())
generator_model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
generator_model.add(BatchNormalization())
generator_model.add(Conv2D(1, (1,1), padding='same', activation='tanh'))
#generator_model.add(Conv2D(3, (1,1), padding='same', activation='tanh')) #FOR RGB (LSUN)

In [7]:
'''Train discriminator for 1 epoch as a start'''
init_data = len(x_train)//200

real = np.random.permutation(x_train)[:init_data]
fake = generator_model.predict(np.random.rand(init_data,100))
x_init = np.concatenate((real,fake))

real_label = np.zeros((init_data,1))
fake_label = np.ones((init_data,1))
y_init = np.concatenate((real_label,fake_label))

'''
figure,axis = plt.subplots(2,3)
plt.tight_layout()
for j in range(3):
    axis[0,j].imshow(real[j],cmap='gray')
    axis[0,j].set_yticklabels([])
    axis[0,j].set_xticklabels([])
    axis[1,j].imshow(fake[j],cmap='gray')
    axis[1,j].set_yticklabels([])
    axis[1,j].set_xticklabels([]
'''

"\nfigure,axis = plt.subplots(2,3)\nplt.tight_layout()\nfor j in range(3):\n    axis[0,j].imshow(real[j],cmap='gray')\n    axis[0,j].set_yticklabels([])\n    axis[0,j].set_xticklabels([])\n    axis[1,j].imshow(fake[j],cmap='gray')\n    axis[1,j].set_yticklabels([])\n    axis[1,j].set_xticklabels([]\n"

In [8]:
discriminator_model.fit(x_init,y_init,batch_size=128,epochs=5)

Epoch 1/5
600/600 [==============================] - 0s - loss: 8.0590     
Epoch 2/5
600/600 [==============================] - 0s - loss: 8.0590     
Epoch 3/5
600/600 [==============================] - 0s - loss: 8.0590     
Epoch 4/5
600/600 [==============================] - 0s - loss: 8.0590     
Epoch 5/5
600/600 [==============================] - 0s - loss: 8.0590     


In [ ]:
model = Sequential([generator_model, discriminator_model])
model.compile(Adam(), loss = "binary_crossentropy")

In [ ]:
real_per_iter = 50
fake_per_iter = 50

real_label = np.zeros((real_per_iter,1))
fake_label = np.ones((fake_per_iter,1))
y_mini = np.concatenate((real_label,fake_label))

discriminator_loss = []
generator_loss = []
num_data = len(x_train)
repeat_discriminator = 1
repeat_generator = 1

In [ ]:
num_iterations = 300
repeat_discriminator = 1
repeat_generator = 1
for i in range(num_iterations):
    print('Iteration #%d' % 
          ((len(discriminator_loss)+(i*repeat_discriminator)-1)))
    real = x_train[np.random.randint(0,high=num_data,size=real_per_iter)]
    fake = generator_model.predict(np.random.rand(fake_per_iter,100))
    x_mini = np.concatenate((real, fake))
    '''Training discriminator'''
    discriminator_model.fit(x_mini,y_mini,batch_size=25,epochs=1)
    #for k in range(repeat_discriminator):
        #discriminator_loss.append(discriminator_model.train_on_batch(x_mini,y_mini))
    
    '''Freeze discriminator'''
    discriminator_model.trainable = False
    for layer in discriminator_model.layers:
        layer.trainable = False
    
    '''Training generator'''
    generator_x = np.random.rand(fake_per_iter*2,100)
    generator_y = np.zeros((fake_per_iter*2,1))
    generator_model.fit(generator_x,generator_y,batch_size=25,epochs=1)
    #for k in range(repeat_generator):
        #generator_loss.append(model.train_on_batch(generator_x, generator_y))
    
    '''Unfreeze discriminator'''
    discriminator_model.trainable = True
    for layer in discriminator_model.layers:
        layer.trainable = True
    
    '''Show losses'''
    #print('Discriminator loss = %.6f Generator loss = %.6f'
          #% (discriminator_loss[len(discriminator_loss)-1],
             #generator_loss[len(generator_loss)-1]))

In [ ]:
figure,axis = plt.subplots(1,2)
plt.tight_layout()
axis[0].plot(discriminator_loss)
axis[0].set_title("Discriminator Loss")
axis[1].plot(generator_loss)
axis[1].set_title("Generator Loss")

In [ ]:
num_generated = 9
in_noise = np.random.rand(num_generated,100)
im = generator_model.predict(in_noise)
figure,axis = plt.subplots(3,3,figsize=(10,10))
plt.tight_layout()
for i in range(3):
    for j in range(3):
        axis[i,j].imshow(im[i*3+j])
        axis[i,j].set_yticklabels([])
        axis[i,j].set_xticklabels([])